In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
## Datasets needed to get production: basedata/VXMD.csv, basedata/VST.csv, basedata/VDFM.csv, basedata/VDPM.csv,
## basedata/VDGM.csv, basedata/

## or just baseview/vom.csv


############### Some important functions to carry around #####################
def save_obj(obj, loc ):
    with open(loc, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(loc):
    with open(loc, 'rb') as f:
        return pickle.load(f)

def reverse_dict(my_map):
    inv_map = {v: k for k, v in my_map.items()}
    return inv_map

def convertRow2Matrix(df, index_cols, maps):
    ''' This is a new function: example use:
    
    vom_np_2 = convertRow2Matrix(vom, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
    
    '''
    df_copy = df.copy()
    index_cols_n = [col+'_Number' for col in index_cols]
    for i, col in enumerate(index_cols):
        df_copy[index_cols_n[i]] = df_copy[col].map(maps[i])
    
    df_mi = df_copy.set_index(index_cols_n)
    df_mi.drop(index_cols, axis=1, inplace=True)
    
    shape = list(map(len, df_mi.index.levels))
    output = np.full(shape, -1000)
    
    for i in list(df_mi.index.values):
        output[i] = df_mi.loc[i].values[0]
        
    print('Number of empty values: ', output[output<0])
    
    return output

############################################################################

# Domestic demand for a produce: VDPMir + VDGMir + SjVDFMijr  
root = '/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/output/basedata/{}.csv'

vdpm =  pd.read_csv(root.format('VDPM'), index_col=0, header=1, low_memory=False)
vdgm =  pd.read_csv(root.format('VDGM'), index_col=0, header=1, low_memory=False)
vdfm =  pd.read_csv(root.format('VDFM'), index_col=0, header=1, low_memory=False)

vipm =  pd.read_csv(root.format('VIPM'), index_col=0, header=1, low_memory=False)
vigm =  pd.read_csv(root.format('VIGM'), index_col=0, header=1, low_memory=False)
vifm =  pd.read_csv(root.format('VIFM'), index_col=0, header=1, low_memory=False)

comm_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')
reg_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')
prodcomm_2_num = load_obj('/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/prodcomm_2_num.pickle')

In [3]:
root = '/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/{}.pickle'
X = load_obj(root.format('X'))
Y = load_obj(root.format('Y'))

In [4]:
vdpm_np = convertRow2Matrix(vdpm, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
vdgm_np = convertRow2Matrix(vdgm, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
vdfm_np = convertRow2Matrix(vdfm, ['TRAD_COMM','PROD_COMM','REG'], [comm_2_num, prodcomm_2_num, reg_2_num])

Number of empty values:  []
Number of empty values:  []
Number of empty values:  []


In [5]:
vipm_np = convertRow2Matrix(vipm, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
vigm_np = convertRow2Matrix(vigm, ['TRAD_COMM','REG'], [comm_2_num, reg_2_num])
vifm_np = convertRow2Matrix(vifm, ['TRAD_COMM','PROD_COMM','REG'], [comm_2_num, prodcomm_2_num, reg_2_num])

Number of empty values:  []
Number of empty values:  []
Number of empty values:  []


In [6]:
del vdpm, vdgm, vdfm, vipm, vigm, vifm

In [7]:
## See Section 4 in the paper. Demand is calibrated using the trade data... But in the data description,
## Check out vim. It seems better for this dataset

In [8]:
# Local Supply for Demand for a good in a country :
vdm = vdpm_np + vdgm_np + vdfm_np.sum(axis=1)

In [9]:
# Imported Supply for Demand in a country :
vim = vipm_np + vigm_np + vifm_np.sum(axis=1)

In [10]:
D = vdm + vim

In [12]:
D.sum(axis=1)

array([  136751,   146417,   220083,   747641,   159321,    48575,
          64233,   297030,   229747,   468730,   283674,    28102,
         212323,   207321,   239652,  1765109,   351158,   682544,
         370885,   442143,   262815,   657513,   154064,   142830,
        1983982,   961830,  1121357,   751020,   366773,   969239,
        1841645,  2933172,  4971856,  1253501,  1856539,  1110737,
        1825925,  3210606,   944758,  2608844,  5389349,   984617,
        2079392,   270288,   364127,  8286448, 10297559,  3869466,
         346337,   737571,  2107156,  3951544,  1426810, 11226918,
        3208434, 13715612,  3249377])

In [21]:
root_matrix = '/Users/boraozaltun/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/D.pickle'
save_obj(D, root_matrix)

In [18]:
(Y.sum(axis=1) - D.sum(axis=1))

array([    346,   -4174,   -3108,  -19148,   -5904,     475,    -412,
         -4511,     612,     -62,     920,    -298,    -538,   -1557,
         -6877,  -85923,  -26718,  -55889,   -5159,   -6115,  -10281,
         -5256,   -4184,   -3991,  -30029,  -10984,  -41225,  -43003,
        -15554,  -25815,  -16843,  -50146, -112992,  -21203,  -33523,
        -14968,  -20405,  -76053,  -11748,  -37422, -114077,  -13290,
          4023,    1491,    2569,    3421,    4979,  162536,  317368,
         74473,    4228,    4786,    4097,    5765,    3542,    3469,
           966])

In [20]:
(Y.sum().sum() - D.sum().sum())/Y.sum().sum()

-0.0031359733571236226